In [1]:
import os
import PIL
import re
from db import *
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
from google.cloud import translate_v2 as translate


os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "C:\\Users\\joman\\anaconda3\\envs\\ocr2\\google_key.json"

In [2]:
from google.cloud import translate_v2 as translate
translate_client = translate.Client()
result = translate_client.translate('hi', target_language='th')
print(result["translatedText"])

สวัสดี


## Calling from DB to get raw text data

In [3]:
def get_raw_text():          ## add in parameters to limit the amt of raw text
    qstr = f"""
    select raw_text from aimbox 
    where outlet_id=94
    limit 10
    """
    return aimbox.query(text=qstr)

data = get_raw_text()
data

,raw_text
0,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
1,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
2,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
3,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
4,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
5,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
6,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
7,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
8,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
9,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...


In [4]:
font_dir='font/'
fonts = []

for root, directories, files in os.walk(font_dir):
        for filename in files:
            filepath = root+'\\'+filename
            name = filename.replace(' ','_')
            new=root+'\\'+name
            fonts.append(name)
            os.rename(filepath, new)
fonts

['Athiti_Medium.ttf',
 'ChulaNarak_Regular.ttf',
 'Garuda.ttf',
 'TH_Sarabun_New_Regular.ttf']

In [6]:
# from google.cloud import translate_v2 as translate
# translate_client = translate.Client()
# result = translate_client.translate('hi', target_language='th')
# print(result["translatedText"])

In [28]:
def translate_to_th_img(data, text_dir, img_dir, font_dir, labels, img_name, fonts, status): #name inclusive of txt
    temp = data.split('\n') #change depending on cols of data.
    f= open(text_dir+img_name,"w+", encoding="utf-8")
    for x in range(len(temp)):
        if temp[x] != '':
            try:
                translation = translate_client.translate(str(temp[x]), target_language='th')
                f.write('\n')
                f.write(translation["translatedText"])
            except IndexError:
                continue
            except AttributeError:
                continue
    f.close()

    for i in fonts:
        font = ImageFont.truetype(font_dir+str(i), 25)
        img_loc=img_dir+img_name+'_'+i.split('.')[0]
        print(img_name+'_'+i.split('.')[0]+'.png', translation["translatedText"])
        print(img_loc)
        labels=labels.append({'filename':img_loc+'.png', 'words': data.replace('\n', ' ')}, ignore_index=True)  #removing \n

        with open(text_dir+img_name, encoding="utf8") as f:
            lines = f.readlines()
            size = os.path.getsize(text_dir+img_name)
        empt=''
        string=empt.join(lines)
        img = Image.new(mode='RGB', size=(1000, int(size*0.7)), color='white')
        d = ImageDraw.Draw(img)
        d.text((100, 20), string, fill=(0, 0, 0), font=font)
        img.save(img_loc+".png", "PNG")
    return labels

In [29]:
directory='C:/Users/joman/Documents/GitHub/EasyOCR/trainer/text/'
train_dir = 'C:/Users/joman/Documents/GitHub/EasyOCR/trainer/all_data/data_train/'
valid_dir = 'C:/Users/joman/Documents/GitHub/EasyOCR/trainer/all_data/data_valid/'
font_dir='C:/Users/joman/Documents/GitHub/EasyOCR/trainer/font/'
labels_train = pd.DataFrame(columns=['filename', 'words'])
labels_test = pd.DataFrame(columns=['filename', 'words'])
labels_valid = pd.DataFrame(columns=['filename', 'words'])

In [30]:
for i in range(len(data)):
    valid = round(len(data)*0.15)
    train = len(data) - valid
    if i<=valid:
        labels_valid=translate_to_th_img(data['raw_text'][i], directory, valid_dir, font_dir, labels_valid, 'thai'+str(i), fonts, 'test')
    else:
        labels_train=translate_to_th_img(data['raw_text'][i], directory, train_dir, font_dir, labels_train, 'thai'+str(i), fonts, 'train')



labels_train.to_csv('all_data/data_train/labels.csv', index=False)
labels_valid.to_csv('all_data/data_valid/labels.csv', index=False)
print('done')

thai0_Athiti_Medium.png ,1615358026
C:/Users/joman/Documents/GitHub/EasyOCR/trainer/all_data/data_valid/thai0_Athiti_Medium
thai0_ChulaNarak_Regular.png ,1615358026
C:/Users/joman/Documents/GitHub/EasyOCR/trainer/all_data/data_valid/thai0_ChulaNarak_Regular
thai0_Garuda.png ,1615358026
C:/Users/joman/Documents/GitHub/EasyOCR/trainer/all_data/data_valid/thai0_Garuda
thai0_TH_Sarabun_New_Regular.png ,1615358026
C:/Users/joman/Documents/GitHub/EasyOCR/trainer/all_data/data_valid/thai0_TH_Sarabun_New_Regular
thai1_Athiti_Medium.png ,1594456795
C:/Users/joman/Documents/GitHub/EasyOCR/trainer/all_data/data_valid/thai1_Athiti_Medium
thai1_ChulaNarak_Regular.png ,1594456795
C:/Users/joman/Documents/GitHub/EasyOCR/trainer/all_data/data_valid/thai1_ChulaNarak_Regular
thai1_Garuda.png ,1594456795
C:/Users/joman/Documents/GitHub/EasyOCR/trainer/all_data/data_valid/thai1_Garuda
thai1_TH_Sarabun_New_Regular.png ,1594456795
C:/Users/joman/Documents/GitHub/EasyOCR/trainer/all_data/data_valid/thai1_TH_

In [10]:
# with open('C:/Users/joman/Desktop/easyocr/final_to_upload/gt_train.txt', 'w+', encoding="utf8") as f:
#     labels_train = pd.read_csv(train_dir+'labels.csv')
#     for i in range(len(labels_train)):
#         f.write(labels_train['filename'][i])
#         f.write('\t')
#         f.write(labels_train['raw_text'][i])
#         f.write('\n')
#     f.close()

# with open('C:/Users/joman/Desktop/easyocr/final_to_upload/gt_test.txt', 'w+', encoding="utf8") as f:
#     labels_test = pd.read_csv(test_dir+'labels.csv')
#     for i in range(len(labels_test)):
#         f.write(labels_test['filename'][i])
#         f.write('\t')
#         f.write(labels_test['raw_text'][i])
#         f.write('\n')
#     f.close()
    
# with open('C:/Users/joman/Desktop/easyocr/final_to_upload/gt_valid.txt', 'w+', encoding="utf8") as f:
#     labels_valid = pd.read_csv(valid_dir+'labels.csv')
#     for i in range(len(labels_valid)):
#         f.write(labels_valid['filename'][i])
#         f.write('\t')
#         f.write(labels_valid['raw_text'][i])
#         f.write('\n')
#     f.close()

In [11]:
labels_test['filename'][1]

'C:/Users/joman/Desktop/easyocr/img/test/img/thai0_Aksaramatee_Italic.png'

## Single line text image

In [33]:
data

,raw_text
0,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
1,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
2,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
3,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
4,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
5,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
6,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
7,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
8,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
9,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...


In [5]:
curr = data[0:3]
curr

,raw_text
0,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
1,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
2,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...


Every line is an image by itself. --> split acc to '\n' and run translate_to_th

In [6]:
curr['raw_text'][0].split('\n')[0]

translatessss = translate.Client()
result = translatessss.translate(curr['raw_text'][0].split('\n')[0], target_language='th')
print(result["translatedText"])

แฟรงกี้ส์ นิวยอร์ก วิงส์ อิงค์


In [57]:
def translate_to_th_img_single(data, text_dir, img_dir, font_dir, labels, img_name, fonts, status): #name inclusive of txt
    string=re.sub(r'[\W_]+', ' ', data)#.replace(' ', '') ## removing non-alphanumeric and spaces in the sentence
    if string != ' ' and string != '':
        f= open(text_dir+img_name,"w+", encoding="utf-8")
        try:
            translation = translate_client.translate(string, target_language='th')
            f.write(translation["translatedText"])
        except IndexError:
            pass
        except AttributeError:
            pass
        f.close()
        for i in fonts:
            # print(font_dir + str(i))
            font = ImageFont.truetype(font_dir+str(i), 25)
            name = img_name+'_'+i.split('.')[0]
            img_loc=img_dir+name
            print(img_loc, name, translation["translatedText"])
            labels=labels.append({'filename':name+'.png', 'words': translation["translatedText"]}, ignore_index=True)  #removing \n

            with open(text_dir+img_name, encoding="utf8") as f:
                lines = f.readlines()
            empt=''
            string=empt.join(lines)
            # print(string)
            img = Image.new(mode='RGB', size=(1000, 100), color='white')
            d = ImageDraw.Draw(img)
            d.text((50, 20), string, fill=(0, 0, 0), font=font)
            img.save(img_dir+name+".png", "PNG")
    return labels

In [16]:
fonts_temp = fonts[:1]
fonts_temp

['Athiti_Medium.ttf']

In [35]:
cd C:/Users/joman/Documents/GitHub/EasyOCR/trainer

C:\Users\joman\Documents\GitHub\EasyOCR\trainer


In [58]:
single_text_dir = 'text/'
single_valid_img = 'all_data/data_valid/'
single_train_img = 'all_data/data_train/'
font_dir='font/'

single_labels_valid = pd.DataFrame(columns=['filename', 'words'])
single_labels_train = pd.DataFrame(columns=['filename', 'words'])

In [56]:
tempo=curr['raw_text'][0].split('\n')
tempo[14]

'----------------------------------------'

In [59]:
for i in range(len(curr)): 
    temp=curr['raw_text'][i].split('\n')
    valid = round(len(temp)*0.15)
    for x in range(len(temp)):
        if temp[x] != ' ' and temp[x] != '':
            if x<valid:
                single_labels_valid = translate_to_th_img_single(temp[x], single_text_dir, single_valid_img, font_dir, single_labels_valid, 'thai'+str(i)+'_line'+str(x), fonts_temp, 'valid')
            else: #train_train
                single_labels_train = translate_to_th_img_single(temp[x], single_text_dir, single_train_img, font_dir, single_labels_train, 'thai_'+str(i)+'_line'+str(x), fonts_temp, 'train')

                    
                    
single_labels_valid.to_csv('all_data/data_valid/labels.csv', index=False)
single_labels_train.to_csv('all_data/data_train/labels.csv', index=False)
print('done')            

all_data/data_valid/thai0_line0_Athiti_Medium thai0_line0_Athiti_Medium แฟรงกี้ เอส นิวยอร์ก วิงส์ อิงค์
all_data/data_valid/thai0_line1_Athiti_Medium thai0_line1_Athiti_Medium เป็นเจ้าของ Optd โดย Frankie s NY Wings Inc
all_data/data_valid/thai0_line2_Athiti_Medium thai0_line2_Athiti_Medium FS 8 ระดับ 4 SM Aura
all_data/data_valid/thai0_line3_Athiti_Medium thai0_line3_Athiti_Medium Bonifacio Global City
all_data/data_valid/thai0_line4_Athiti_Medium thai0_line4_Athiti_Medium ป้อม Bonifacio Taguig City
all_data/data_valid/thai0_line5_Athiti_Medium thai0_line5_Athiti_Medium ทะเบียนภาษีมูลค่าเพิ่ม TIN 009 074 986 00010
all_data/data_valid/thai0_line6_Athiti_Medium thai0_line6_Athiti_Medium ขั้นต่ำ 15091811250923388
all_data/data_valid/thai0_line7_Athiti_Medium thai0_line7_Athiti_Medium MSN 8880115010003
all_data/data_valid/thai0_line8_Athiti_Medium thai0_line8_Athiti_Medium ใบเสร็จทางการ
all_data/data_train/thai_0_line9_Athiti_Medium thai_0_line9_Athiti_Medium พิมพ์ซ้ำ
all_data/data_train

In [31]:
# single_labels_train = single_labels_ttest.append(single_labels_tvalid).append(single_labels_ttrain).rename(columns={'raw_text':'words'})
# single_labels_train
# single_labels_train.to_csv('C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training/singlelinetemp\\single_test\\labels.csv', index=False)

In [26]:
# with open('C:\\Users\\joman\\Desktop\\easyocr\\single_line\\data_test\\gt_test.txt', 'w+', encoding="utf8") as f:
#     for i in range(len(single_labels_test)):
#         f.write(single_labels_test['filename'][i])
#         f.write('\t')
#         f.write(single_labels_test['words'][i])
#         f.write('\n')
#     f.close()
    
# with open('C:\\Users\\joman\\Desktop\\easyocr\\single_line\\data_valid\\gt_valid.txt', 'w+', encoding="utf8") as f:
#     for i in range(len(single_labels_valid)):
#         f.write(single_labels_valid['filename'][i])
#         f.write('\t')
#         f.write(single_labels_valid['words'][i])
#         f.write('\n')
#     f.close()
    

# with open('C:\\Users\\joman\\Desktop\\easyocr\\single_line\\data_train\\gt_ttrain.txt', 'w+', encoding="utf8") as f:
#     for i in range(len(single_labels_ttrain)):
#         f.write(single_labels_ttrain['filename'][i])
#         f.write('\t')
#         f.write(single_labels_ttrain['words'][i])
#         f.write('\n')
#     f.close()
    
# with open('C:\\Users\\joman\\Desktop\\easyocr\\single_line\\data_train\\gt_tvalid.txt', 'w+', encoding="utf8") as f:
#     for i in range(len(single_labels_tvalid)):
#         f.write(single_labels_tvalid['filename'][i])
#         f.write('\t')
#         f.write(single_labels_tvalid['words'][i])
#         f.write('\n')
#     f.close()
    
# with open('C:\\Users\\joman\\Desktop\\easyocr\\single_line\\data_train\\gt_ttest.txt', 'w+', encoding="utf8") as f:
#     for i in range(len(single_labels_ttest)):
#         f.write(single_labels_ttest['filename'][i])
#         f.write('\t')
#         f.write(single_labels_ttest['words'][i])
#         f.write('\n')
#     f.close()

In [35]:
char_loaded = open('C:\\Users\\joman\\Desktop\\easyocr\\characters.txt', "r", encoding="utf-8")
lines = char_loaded.readlines()
lines

['0123456789abcdefghijklmnopqrstuvwxyzกขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮอะอิอึอุเอะแอะโอะเอาะเออะเอียะเอือะอัวะอำไอใอเอาอาอีอือูเอแอโอออเออเอียเอืออัว']

## CLI for creation of lmdb dataset

In [36]:
!cd C:/Users/joman/Documents/GitHub/deep-text-recognition-benchmark

In [27]:
!python C:/Users/joman/Documents/GitHub/deep-text-recognition-benchmark/create_lmdb_dataset.py \
--inputPath C:/Users/joman/Desktop/easyocr/single_line/data_test/ \
--gtFile C:/Users/joman/Desktop/easyocr/single_line/data_test/gt_test.txt \
--outputPath C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/evaluation/singlelineeval/

Created dataset with 28 samples


In [28]:
!python C:/Users/joman/Documents/GitHub/deep-text-recognition-benchmark/create_lmdb_dataset.py \
--inputPath C:/Users/joman/Desktop/easyocr/single_line/data_valid/ \
--gtFile C:/Users/joman/Desktop/easyocr/single_line/data_valid/gt_valid.txt \
--outputPath C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/validation/

Created dataset with 30 samples


In [29]:
!python C:/Users/joman/Documents/GitHub/deep-text-recognition-benchmark/create_lmdb_dataset.py \
--inputPath C:/Users/joman/Desktop/easyocr/single_line/data_train/ \
--gtFile C:/Users/joman/Desktop/easyocr/single_line/data_train/gt_ttest.txt \
--outputPath C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training/singlelinetemp/single_test/

Created dataset with 22 samples


In [30]:
!python C:/Users/joman/Documents/GitHub/deep-text-recognition-benchmark/create_lmdb_dataset.py \
--inputPath C:/Users/joman/Desktop/easyocr/single_line/data_train/ \
--gtFile C:/Users/joman/Desktop/easyocr/single_line/data_train/gt_ttrain.txt \
--outputPath C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training/singlelinetemp/single_train/

Created dataset with 77 samples


In [31]:
!python C:/Users/joman/Documents/GitHub/deep-text-recognition-benchmark/create_lmdb_dataset.py \
--inputPath C:/Users/joman/Desktop/easyocr/single_line/data_train/ \
--gtFile C:/Users/joman/Desktop/easyocr/single_line/data_train/gt_tvalid.txt \
--outputPath C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training/singlelinetemp/single_valid/ 

Created dataset with 22 samples


In [42]:
!python C:/Users/joman/Documents/GitHub/deep-text-recognition-benchmark/train.py \
--train_data C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training \
--valid_data C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/validation \
--select_data singlelinetemp \
--batch_ratio 1 \
--Transformation None \
--FeatureExtraction VGG \
--SequenceModeling BiLSTM \
--Prediction CTC \
--data_filtering_off

device count 0

Traceback (most recent call last):
  File "C:\Users\joman\Documents\GitHub\deep-text-recognition-benchmark\train.py", line 348, in <module>
    train(opt)
  File "C:\Users\joman\Documents\GitHub\deep-text-recognition-benchmark\train.py", line 234, in train
    print(lines)
  File "C:\Users\joman\anaconda3\envs\ocr2\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode characters in position 0-7: character maps to <undefined>



training
--------------------------------------------------------------------------------
dataset_root: C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training
opt.select_data: ['singlelinetemp']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training	 dataset: singlelinetemp
sub-directory:	/singlelinetemp\single_test	 num samples: 22
sub-directory:	/singlelinetemp\single_train	 num samples: 77
sub-directory:	/singlelinetemp\single_valid	 num samples: 22
num total samples of singlelinetemp: 121 x 1.0 (total_data_usage_ratio) = 121
num samples of singlelinetemp per batch: 192 x 1.0 (batch_ratio) = 192
--------------------------------------------------------------------------------
Total_batch_size: 192 = 192
--------------------------------------------------------------------------------
dataset_root:    C:/Users/joman/De

## Getting gt_file for images

In [38]:
directory = 'C:\\Users\\joman\\Desktop\\easyocr\\data_english_test\\training\\eval'

In [39]:
#getting directory of every image
import os

def get_gt(directory):
    file_paths = []  # List which will store all of the full filepaths.
    with open('C:\\Users\\joman\\Desktop\\easyocr\\data_english_test\\training\\gt_train_eval.txt', 'w+', encoding="utf8") as f:
    # Walk the tree.
        for root, directories, files in os.walk(directory):
            for filename in files:
                # Join the two strings in order to form the full filepath.
                filepath = root+'\\'+filename
                name = filename.split('_')[0]
                if name.isalpha()==True:
                    f.write(filepath)
                    f.write('\t')
                    f.write(name)
                    f.write('\n')
                else:
                    os.remove(filepath)

In [40]:
get_gt(directory)